In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install mtcnn

In [0]:
import pickle
import keras
from keras.models import load_model
import cv2
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
from mtcnn import MTCNN

In [0]:
folder_path = '/content/drive/My Drive/week10/face_detection'
detector = MTCNN()
feature_extractor = load_model(os.path.join(folder_path, 'facenet.h5'))
model_age = load_model(os.path.join(folder_path, 'age100.h5'))
model_gender = load_model(os.path.join(folder_path, 'gender100.h5'))
model_emotion = load_model(os.path.join(folder_path, 'emotion48.h5'))

In [0]:
# read db
with open(os.path.join(folder_path, 'db.pickle'), 'rb') as file:
    db = pickle.load(file)
embeddings = db['embeddings']
names = db['names']

In [0]:
# detect face
def detect_faces(img):
    face_imgs = []
    results = detector.detect_faces(img)
    # extract the bounding box from the first face
    print('# of faces: ', len(results))
    for i in range(len(results)):
        x1, y1, width, height = results[i]['box']
        x2, y2 = x1 + width, y1 + height
        patch = img[y1:y2, x1:x2] # crop face
        face_imgs.append(patch)
    return face_imgs

In [0]:
# Standardization
def preprocess(imgs): 
    for i in range(imgs.shape[0]):
        # standardization
        img = imgs[i]
        mean, std = img.mean(), img.std()
        img = (img - mean) / std
        imgs[i] = img
    return imgs
# Normalization
def normalize(img):
    return img / 255.

In [0]:
def euclidean_distance(x, y):
    sum_square = np.sum(np.square(x - y), keepdims=True)
    return np.sqrt(sum_square)

In [0]:
def predict_age(img):
    img_size = 100
    img = normalize(img)
    img = cv2.resize(img, (img_size, img_size))
    model_input = np.zeros((1, img_size, img_size, 3))
    model_input[0] = img
    ages = model_age.predict(model_input)
    print('age: ', ages[0])
    return 
    
def predict_emotion(img):
    img_size = 48
    cls_map = ['Angry', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral', 'Disgust']
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = normalize(img)
    img = cv2.resize(img, (img_size, img_size))
    model_input = np.zeros((1, img_size, img_size, 1))
    model_input[0] = np.expand_dims(img, axis=-1) # 48,48 -> 48,48,1
    emotions_pred = model_emotion.predict(model_input)
    emotion_pred = emotions_pred[0]
    emotion_name = cls_map[np.argmax(emotion_pred)]
    print('emotion: ', emotion_name)
    return
def predict_gender(img):
    img_size = 100
    img = normalize(img)
    img = cv2.resize(img, (img_size, img_size))
    model_input = np.zeros((1, img_size, img_size, 3))
    model_input[0] = img
    genders = model_gender.predict(model_input)
    gender = genders[0]
    if gender > 0.5:
        print('Male')
    else:
        print('Female')
    return

In [0]:
def face_id(filename, IMG_SIZE=160):
    raw_img = cv2.imread(os.path.join(folder_path, filename))[:,:,::-1]
    faces = detect_faces(raw_img)
    if len(faces) == 0:
        print('No face')
        return
    else:
        # get face embeddings
        face = faces[0]
        # More predictions
        predict_age(face)
        predict_emotion(face)
        predict_gender(face)
        # ID
        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        model_input = np.zeros((1, IMG_SIZE, IMG_SIZE, 3))
        model_input[0] = face
        model_input = preprocess(model_input)
        query_embeddings = feature_extractor.predict(model_input)
        query_embedding = query_embeddings[0]
        
        # compute distance
        distances = np.zeros((len(embeddings)))
        for i, embed in enumerate(embeddings):
            distance = euclidean_distance(embed, query_embedding)
            distances[i] = distance

        # find min distance    
        idx_min = np.argmin(distances)
        distance, name = distances[idx_min], names[idx_min]
        print('name: ', name, ' distance: ',distance)

In [0]:
path = 'face1.jpg'
face_id(path)
plt.imshow(cv2.imread(os.path.join(folder_path, path))[:,:,::-1])